In [185]:
import pandas as pd
from os import listdir
import uuid
from datetime import date
import numpy as np
import re
from datetime import datetime
from tqdm import tqdm
from fuzzywuzzy import process

In [7]:
import warnings
warnings.filterwarnings("ignore")

## Functions

In [ ]:
def dummy_facebook_date(x):
    if pd.isna(x):
        return(0)
    else:
        return(1)

def load_africa_post_matches():
    africa = pd.read_excel('../../../data/1-factchecks/1-posts_factchecks/africacheck_matches.xlsx')
    africa = africa[africa['no_socialmedia_factcheck'] == 1][['id_factcheck', 'date_factcheck_facebook']]
    africa['date_factcheck_facebook'] = africa['date_factcheck_facebook'].apply(lambda x: datetime.strptime(re.sub(' CST| CDT', '', x), '%Y-%m-%d %H:%M:%S').date())
    africa['post_facebook_factcheck'] = None
    return(africa)


def load_latam_post_matches():
    latam = pd.read_excel('../../../data/1-factchecks/1-posts_factchecks/factchekers_check.xlsx')
    latam = latam[latam['socialmedia_factcheck'] == 1][['id_factcheck', 'post_factcheck','date_factcheck_facebook']]
    latam = latam.rename(columns = {'post_factcheck':'post_facebook_factcheck'})
    latam['date_factcheck_facebook'] = latam['date_factcheck_facebook'].apply(lambda x: datetime.strptime(re.sub(' CST| CDT', '', x), '%Y-%m-%d %H:%M:%S').date())    
    return(latam)

def load_post_matches():
    africa = load_africa_post_matches()
    latam = load_latam_post_matches()
    return(pd.concat([africa, latam]))

In [11]:
def assign_date_partnership(organizacion):
    dict_poynters = {'africa_check':datetime.date(2019, 8, 15), 
                    'la_silla_vacia':datetime.date(2020, 1, 31),
                     'chequeado':datetime.date(2018, 5, 1),
                     'animalpolitico':datetime.date(2018, 3, 1),
                     'colombiacheck': datetime.date(2020, 9, 28),
                     'ecuador_chequea':datetime.date(2020, 4, 1),
                     'larepublica':datetime.date(2020, 6, 1),
                    }
    
    if organizacion in dict_poynters.keys():
        return(dict_poynters[organizacion])
    else:
        return(np.nan)

def poynter_any_time(organizacion):
    dict_poynters = {'africa_check':datetime.date(2019, 8, 15), 
                    'la_silla_vacia':datetime.date(2020, 1, 31),
                     'chequeado':datetime.date(2018, 5, 1),
                     'animalpolitico':datetime.date(2018, 3, 1),
                     'colombiacheck': datetime.date(2020, 9, 28),
                     'ecuador_chequea':datetime.date(2020, 4, 1),
                     'larepublica':datetime.date(2020, 6, 1),
                    }
    
    if organizacion in dict_poynters.keys():
        return(1)
    else:
        return(0)

In [12]:
def include_missing_data():
    newspapers = pd.read_excel('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/13-fact-checks_virality/00-raw/missing_factchecks_links_newspapers.xlsx')
    social = pd.read_excel('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/13-fact-checks_virality/00-raw/missing_factchecks_links_social.xlsx')
    data = pd.concat([newspapers, social])
    data = data[~data['link_chequeo'].isna()]
    data['id_chequeo'] = data.apply(lambda x: str(uuid.uuid3(uuid.NAMESPACE_URL,x['link_chequeo'])), axis = 1)
    return(data)

In [15]:
def load_factchecks_newspapers():
    path = '/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/04-fake_news_dataset/newspapers/Fact-Checkers/'
    data = []
    for i in listdir(path):
        try:
            data.append(pd.read_excel(path + i))
        except:
            pass
        
    factchecks = pd.concat(data)
    extra = include_missing_data()
    
    ## include missing factchecks
    factchecks = factchecks[~factchecks['id_desinformacion'].isin(list(extra.id_desinformacion))]
    factchecks = pd.concat([factchecks, extra]).drop(['Unnamed: 0'], axis = 1)
    factchecks = factchecks[['id_chequeo', 'organizacion', 'pais', 'titulo_chequeo', 'fecha_chequeo',
       'link_chequeo', 'id_desinformacion']]
    factchecks['facebook_partnership_date'] = factchecks.apply(lambda x: assign_date_partnership(x['organizacion']), axis = 1)
    factchecks['facebook_partnership'] = factchecks.apply(lambda x: poynter_any_time(x['organizacion']), axis = 1)    
    return(factchecks)


def load_factchecks_social():
    path = '/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/04-fake_news_dataset/social/Fact-Checkers/'
    data = []
    for i in listdir(path):
        try:
            data.append(pd.read_excel(path + i))
        except:
            pass
        
    factchecks = pd.concat(data)
    extra = include_missing_data()
    
    ## include missing factchecks
    factchecks = factchecks[~factchecks['id_desinformacion'].isin(list(extra.id_desinformacion))]
    factchecks = pd.concat([factchecks, extra]).drop(['Unnamed: 0'], axis = 1)
    factchecks = factchecks[['id_chequeo', 'organizacion', 'pais', 'titulo_chequeo', 'fecha_chequeo',
       'link_chequeo', 'id_desinformacion']]
    factchecks['facebook_partnership_date'] = factchecks.apply(lambda x: assign_date_partnership(x['organizacion']), axis = 1)
    factchecks['facebook_partnership'] = factchecks.apply(lambda x: poynter_any_time(x['organizacion']), axis = 1)    
    return(factchecks)


def load_factchecks():
    social = load_factchecks_social()
    social['type_factcheck'] = 'social_media'
    newspapers = load_factchecks_newspapers()
    newspapers['type_factcheck'] = 'medios_tradicionales'
    africa = include_africa_check().rename(columns = {'source_type':'type_factcheck'})[list(newspapers.columns)]
    
    all_data = pd.concat([social, newspapers, africa])
    all_data = all_data.drop_duplicates()
    all_data = all_data.rename(columns = {'id_chequeo':'id_factcheck'})
    return(all_data)


In [4]:
def include_africa_check():
    africa_check = pd.read_excel('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/06-virality/africa/fact_checks/fact_checks_clean.xlsx')
    africa_check['pais'] = 'africa'
    africa_check['organizacion'] = 'africa_check'
    africa_check['titulo_chequeo'] = africa_check.apply(lambda x: obtain_title_chequeo(x['link_chequeo']), axis = 1)
    africa_check['facebook_partnership_date'] = africa_check.apply(lambda x: assign_date_partnership(x['organizacion']), axis = 1)
    africa_check['facebook_partnership'] = africa_check.apply(lambda x: poynter_any_time(x['organizacion']), axis = 1)    
    
    africa_check = africa_check.rename(columns = {'id_factcheck':'id_chequeo', 
                                                 'date_factcheck':'fecha_chequeo'})
    return(africa_check)

In [9]:
def obtain_title_chequeo(link):
    link = re.sub('[/]$', '', link).split('/')[-1:][0]
    link = re.sub('-', ' ', link)
    return(link)

## Apply functions

In [17]:
all_data = load_factchecks()

In [140]:
all_data.head()

id_factcheck organizacion  pais  \
0  2c51a443-dcfe-3fdc-b901-79566bb18dd0  larepublica  peru   
1  1718b0ab-d593-3704-a78a-bb095d6ecb14  larepublica  peru   
2  3c167832-8e73-330a-ab8a-19b219f79dfc  larepublica  peru   
3  656ae72f-a364-353d-813a-dd24472479a6  larepublica  peru   
4  6fcce096-cf30-3f7c-ae86-591590d024e6  larepublica  peru   

                                      titulo_chequeo        fecha_chequeo  \
0  cafe haiti es falso que el cafe haiti de miraf...  2020-05-08 00:00:00   
1  areas de aislamiento es falsa la publicacion d...  2020-05-08 00:00:00   
2  dioxido de cloro es imprecisa la publicacion d...  2020-04-08 00:00:00   
3  5g es falso que foto es de vacuna china con ar...  2020-03-08 00:00:00   
4  coronavirus es impreciso afirmar que colegio m...  2020-03-08 00:00:00   

                                        link_chequeo id_desinformacion  \
0  https://larepublica.pe/verificador/2020/08/05/...               NaN   
1  https://larepublica.pe/verificador/2020/08/05/...               NaN   
2  https://larepublica.pe/verificador/2020/08/04/...               NaN   
3  https://larepublica.pe/verificador/2020/08/03/...               NaN   
4  https://larepublica.pe/verificador/2020/08/03/...               NaN   

  facebook_partnership_date  facebook_partnership type_factcheck  
0                2020-06-01                     1   social_media  
1                2020-06-01                     1   social_media  
2                2020-06-01                     1   social_media  
3                2020-06-01                     1   social_media  
4                2020-06-01                     1   social_media

## Search for Facebook Post

In [107]:
matches_post = load_post_matches()

In [122]:
factchecks_data = pd.merge(left = all_data, 
        right = matches_post, 
        on = 'id_factcheck', 
        how = 'left').drop_duplicates()

In [127]:
factchecks_data = factchecks_data[['id_factcheck', 'type_factcheck','organizacion', 'pais',
                 'facebook_partnership_date', 'facebook_partnership',
                'titulo_chequeo', 'link_chequeo', 
                 'fecha_chequeo', 'post_facebook_factcheck', 'date_factcheck_facebook', 
                'id_desinformacion']].rename(columns = {'post_facebook_factcheck':'link_facebook_factcheck'})

In [135]:
factchecks_data['dummy_facebook_date'] = factchecks_data['date_factcheck_facebook'].apply(lambda x: dummy_facebook_date(x))

In [133]:
def dummy_facebook_date(x):
    if pd.isna(x):
        return(0)
    else:
        return(1)

In [136]:
factchecks_data[['id_factcheck', 'dummy_facebook_date']].groupby('dummy_facebook_date').count()

id_factcheck
dummy_facebook_date              
0                            3844
1                             330

In [139]:
len(factchecks_data)

4189

## obtain difference between factcheck date and facebook date

In [202]:
def clean_dates(x):
    if pd.isna(x) or x == 44106:
        return(None)
    else:
        if isinstance(x, datetime.date):
            return(x)
        else:
            if any(re.findall('/', x)):
                if len(x.split('/')[2]) == 2:
                    month_ = int(x.split('/')[0])
                    day_ = int(x.split('/')[1])
                    year_ = int('20' + x.split('/')[2])
                    return(datetime.date(year_, month_, day_))
                else:
                    try:
                        month_ = int(x.split('/')[1])
                        day_ = int(x.split('/')[0])
                        year_ = int(x.split('/')[2])
                        return(datetime.date(year_, month_, day_))
                    except:
                        month_ = int(x.split('/')[1])
                        day_ = int(x.split('/')[0])
                        year_ = int(x.split('/')[2])
                        return(datetime.date(year_,  day_, month_))                        
                        
            else:
                
                elements = x.split('T')[0]
                return(datetime.date(int(elements.split('-')[0]), 
                                    int(elements.split('-')[1]), 
                                    int(elements.split('-')[2])))

In [225]:
def obtain_diff_dates(x, y):
    if pd.isna(x) or pd.isna(y):
        return(None)
    else:
        try:
            return((x -y.date()).days)
        except:
            return((x -y).days)

In [204]:
import datetime
factchecks_data['fecha_chequeo'] = factchecks_data['fecha_chequeo'].apply(lambda x: clean_dates(x))

In [226]:
factchecks_data['difference_dates'] = factchecks_data.apply(lambda x: obtain_diff_dates(x['date_factcheck_facebook'], 
                                                                                       x['fecha_chequeo']), axis = 1)

In [238]:
factchecks_data['difference_dates'] = np.where(factchecks_data['difference_dates'] == 2832.0, None, factchecks_data['difference_dates'])

In [246]:
factchecks_data.to_excel('../../../data/1-factchecks/2-clean_factchecks/factchecks_data.xlsx')

In [82]:
def clean_date_factcheck(x):
    try:
        return(datetime.strptime(re.sub(' CST| CDT', '', x), '%Y-%m-%d %H:%M:%S').date())
    except:
        return(None)
            

In [38]:
def load_data_posts():
    path = '/Users/cblanesg/misinformation_socialmedia/data/1-factchecks/1-posts_factchecks/posts/'
    posts_facebook = []
    for i in listdir(path):
        try:
            df = pd.read_csv(path + i)
            df['organizacion'] = re.sub('[.]csv','', i)
            posts_facebook.append(df)
        except:
            print(i)

    posts_facebook = pd.concat(posts_facebook)
    return(posts_facebook)

In [71]:
def match_data_function(data_factchecks, posts_facebook):
    
    data_factchecks = data_factchecks[['id_factcheck', 'organizacion', 
        'link_chequeo', 'fecha_chequeo', 'id_desinformacion']]
    data_factchecks = data_factchecks.drop_duplicates()
    data_factchecks['date_factcheck_facebook'] = None
    
    
    matches_data = []

    for i in tqdm(list(data_factchecks.organizacion.unique())):
        if i == 'Chequeado-LATAM':
            df = data_factchecks[data_factchecks['organizacion'] == 'Chequeado-LATAM']
            df_posts = posts_facebook[posts_facebook['organizacion'] == 'chequeado']
            df_posts = clean_links(df_posts)
            input_matches = df_posts[['Final Link', 'URL', 'Created', 'title_link']].dropna()
            out = match_factcheck_w_posts(df, input_matches)
            matches_data.append(out)
        else:        
            df = data_factchecks[data_factchecks['organizacion'] == i]
            df_posts = posts_facebook[posts_facebook['organizacion'] == i]     
            df_posts = clean_links(df_posts)
            input_matches = df_posts[['Final Link', 'URL', 'Created', 'title_link']].dropna()
            out = match_factcheck_w_posts(df, input_matches)
            matches_data.append(out)

    df_out = pd.concat(matches_data)
    return(df_out)

In [72]:
def match_factcheck_w_posts(df, df_posts):
    date_factchecks = []
    match_link = []
    score_match_link = []
    match_title = []
    score_match_title = []
    post_url = []
    
    df = df.reset_index().drop('index', axis = 1)
    df['title_factcheck'] = df.apply(lambda x: title_factcheck_function(x['link_chequeo']), axis = 1)
    for i in tqdm(range(0, len(df))):
        if pd.isna(df.loc[i]['date_factcheck_facebook']):
            link_factcheck = df.loc[i]['link_chequeo']
            
            if pd.isna(link_factcheck):
                    score_match_link.append(None)
                    match_link.append(None)
                    date_factchecks.append(None)
                    match_title.append(None)
                    score_match_title.append(None)
                    post_url.append(None)
            else:
                match, score = process.extractOne(link_factcheck, list(df_posts['Final Link']))
                if score >= 90:
                    score_match_link.append(score)
                    match_link.append(match)
                    date_factchecks.append(list(df_posts[df_posts['Final Link'] == match].Created)[0])
                    match_title.append(None)
                    score_match_title.append(None)
                    post_url.append(list(df_posts[df_posts['Final Link'] == match].URL)[0])
                else:
                    title_factcheck = df.loc[i]['title_factcheck']
                    match, score = process.extractOne(title_factcheck, list(df_posts['title_link']))
                    if score > 80:
                        score_match_link.append(None)
                        match_link.append(None)
                        date_factchecks.append(list(df_posts[df_posts['title_link'] == match].Created)[0])
                        match_title.append(match)
                        score_match_title.append(score)
                        post_url.append(list(df_posts[df_posts['title_link'] == match].URL)[0])
                    else:
                        score_match_link.append(None)
                        match_link.append(None)
                        date_factchecks.append(None)
                        match_title.append(None)
                        score_match_title.append(None)
                        post_url.append(None)
        else:
            score_match_link.append(None)
            match_link.append(None)
            date_factchecks.append(df.loc[i]['date_factcheck_facebook'])
            match_title.append(None)
            score_match_title.append(None)
            post_url.append(None)
            
    df['date_factcheck_facebook'] = date_factchecks
    df['match_link_factcheck'] = match_link
    df['score_match_link_factcheck'] = score_match_link
    df['match_title_factcheck'] = match_title
    df['score_match_title_factcheck'] = score_match_title
    df['post_url'] = post_url
    return(df) 
def clean_links(df):
    final_links = []
    for i in list(df['Final Link']):
        if pd.isna(i):
            final_links.append(np.nan)
        elif any(re.findall('https://www.facebook.com/login/|https://api.whatsapp.com/',i)):
            final_links.append(np.nan)
        else:
            final_links.append(i)
    df['Final Link'] = final_links
    df = df[~df['Final Link'].isna()]
    df = df[df['Final Link'] != 'https://lasillavacia.com/']
    return(df)

def title_factcheck_function(x):
    if pd.isna(x):
        return(None)
    else:
        return(re.sub('[/]$', '', x).split('/')[-1:][0])



In [62]:
latam = all_data[all_data['pais'] != 'africa']
data_posts = load_data_posts()
data_posts['title_link'] = data_posts.apply(lambda x: title_factcheck_function(x['Final Link']), axis = 1)
data_posts['title_link'] = data_posts.apply(lambda x: title_factcheck_function(x['Final Link']), axis = 1)

.DS_Store


In [73]:
match_out = match_data_function(latam, data_posts)

 72%|███████▏  | 155/214 [17:57<09:06,  9.25s/it]


  0%|          | 0/22 [21:03<?, ?it/s]


KeyboardInterrupt: 

## Save Data

In [91]:
all_data.to_excel('/Users/cblanesg/misinformation_socialmedia/data/1-factechecks/0-factchecks/factchecks.xlsx')